In [1]:
from mindnlp.utils import http_get

url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the defa

In [2]:
from mindspore.dataset import TextFileDataset

dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

In [3]:
train_dataset, test_dataset = dataset.split([0.9, 0.1], randomize=False)

In [4]:
# article: [CLS] xxxxx [SEP]
# summary: [CLS] xxxxx [SEP]

In [5]:
import json
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=6, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [6]:
from mindnlp.transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4)

In [8]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]]),
 Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]])]

In [9]:
len(tokenizer)

21128

In [10]:
from mindnlp.core.nn import functional as F
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def construct(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = F.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return loss

In [11]:
num_epochs = 5
learning_rate = 1.5e-4
warmup_steps = 2000
max_grad_norm = 1.0
num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [12]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2ForSummarization(config)

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [13]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [14]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="gpt2_summarization",
    save_steps=train_dataset.get_dataset_size(),
    save_total_limit=3,
    logging_steps=100,
    max_steps=num_training_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_steps=warmup_steps
    
)

In [15]:
from mindnlp.engine import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

  0%|                                                                                               | 0/56250 …

{'loss': 6.8268, 'learning_rate': 7.499999999999999e-06, 'epoch': 0.01}
{'loss': 6.0453, 'learning_rate': 1.4999999999999999e-05, 'epoch': 0.02}
{'loss': 5.2041, 'learning_rate': 2.2499999999999998e-05, 'epoch': 0.03}
{'loss': 4.7843, 'learning_rate': 2.9999999999999997e-05, 'epoch': 0.04}
{'loss': 4.4802, 'learning_rate': 3.75e-05, 'epoch': 0.04}
{'loss': 4.154, 'learning_rate': 4.4999999999999996e-05, 'epoch': 0.05}
{'loss': 3.974, 'learning_rate': 5.2499999999999995e-05, 'epoch': 0.06}
{'loss': 3.838, 'learning_rate': 5.9999999999999995e-05, 'epoch': 0.07}
{'loss': 3.8311, 'learning_rate': 6.75e-05, 'epoch': 0.08}
{'loss': 3.8637, 'learning_rate': 7.5e-05, 'epoch': 0.09}


In [ ]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [ ]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

In [ ]:
model = GPT2LMHeadModel.from_pretrained('./checkpoint/gpt2_summarization_epoch_4.ckpt', config=config)

In [ ]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 10:
        break